In [2]:
!pip install transformers>=4.21.0 --quiet
!pip install torch torchvision torchaudio --quiet
!pip install tqdm --quiet
!pip install datasets --quiet
!pip install datatrove --quiet

try:
    !pip install flash-attn --no-build-isolation --quiet
    print("Flash Attention がインストールされました")
except:
    print("Flash Attention のインストールに失敗 - 標準Attentionを使用します")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 111.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Flash Attention がインストールされました


In [3]:
import torch
import argparse
import time
from transformers import AutoModel, AutoTokenizer
import time

model_name = "asap-bb/mylightlm_sample"
#model_name = "./hf_model"  # ローカルのHuggingFace形式モデルを使用する場合
prompt = "I'm Mike. I live in"
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16 if device == "cuda" else torch.float32
model = AutoModel.from_pretrained(model_name, trust_remote_code=True, dtype=dtype).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.eval()

print(f"Device: {device}")
print(f"Data type: {dtype}")
print(f"Model loaded on {device} with {dtype}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

model.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/asap-bb/mylightlm_sample:
- model.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.67G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

Device: cpu
Data type: torch.float32
Model loaded on cpu with torch.float32


In [5]:
# トークナイズ
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs["input_ids"].to(device)

with torch.no_grad():
    start_time = time.time()
    # use_cacheをTrueに設定してKVキャッシュを使用
    with torch.autocast(device_type=device, dtype=dtype):
        output_ids = model.generate(
            input_ids,
            max_tokens=100,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            use_cache=True,
        )

    end_time = time.time()
    print(f"⏱️ Generation took {end_time - start_time:.2f} seconds")

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(f"\n" + "="*50)
print(f"📄 Generated Text:")
print(f"="*50)
print(generated_text)
print(f"="*50)

⏱️ Generation took 27.00 seconds

📄 Generated Text:
I'm Mike. I live in a small town near the coast of England, on the shores of what is now called the English Channel. It's a narrow bay on the west side of London Harbour at its highest point and it's just over 50 feet below sea level.
But that's not all you need to do is head north for a few hours every day, because when it comes back up into the bay there are a couple hundred people waiting before they're ready to move up to their boats, or they'll
